In [1]:
import weave
from set_env import set_env
import nest_asyncio
from evalforge import EvalForge, convert_datapoint_to_example
from combined_scorer import predict_passthrough
import asyncio

In [2]:
set_env("OPENAI_API_KEY")
set_env("WANDB_API_KEY")

2024-09-19 10:24:20.841 | WARNING  | set_env.set_env:set_env:100 - 
        Unable to set WANDB_API_KEY=WANDB_API_KEY,
        not in colab or Secrets not set, not kaggle
        or Secrets not set, no .env/dotenv/env file
        in the current working dir or parent dirs.


loading envfile='/Users/anishshah/Documents/Manual Library/GitHub(1)/improve-evals/.env' with dotenv_values(envfile)


In [3]:
try:
    import IPython
    in_jupyter = True
except ImportError:
    in_jupyter = False
if in_jupyter:
    nest_asyncio.apply()

In [4]:
import random
weave.init(f"evalgen_test_{random.randint(0, 1000000)}")

Logged in as Weights & Biases user: a-sh0ts.
View Weave data at https://wandb.ai/a-sh0ts/evalgen_test_943788/weave


In [5]:
TEST_TASK = "medical"
if TEST_TASK == "medical":
    all_data = weave.ref("weave:///a-sh0ts/medical_data_results/object/medical_data_annotations:7GcCtWgyPTWtKY48Z7v5VxwCNZXTTTpSMbmubAbyHT8").get()
    data = random.sample(all_data, 10)
elif TEST_TASK == "product":
    pass
else:
    all_data = data = [
        ({"text": "Summarize the impact of climate change on polar bears."}, {"text": "Climate change is reducing sea ice, which polar bears rely on for hunting seals."}, 1, "Accurate and relevant."),
        ({"text": "Explain the process of photosynthesis."}, {"text": "Photosynthesis is the process by which plants use sunlight to synthesize foods from carbon dioxide and water."}, 1, "Correct and detailed."),
        ({"text": "What are the main causes of the American Civil War?"}, {"text": "The main causes were slavery, states' rights, and economic differences."}, 1, "Concise and accurate."),
        ({"text": "Describe the symptoms of COVID-19."}, {"text": "COVID-19 is caused by a virus that originated in bats."}, 0, "Irrelevant and incorrect."),
        ({"text": "What is the significance of the Magna Carta?"}, {"text": "The Magna Carta was a document that limited the power of the king and established certain legal rights."}, 1, "Historically accurate and relevant.")
    ]

In [6]:
forger = EvalForge()

results = await forger.predict(data)

best_criteria:  {}
filtered_criterion_assertion_map:  {}


In [7]:
forged_judge = results["forged_judges"]["judge"]
forged_judges_metrics = results["forged_judges"]["alignment_metrics"]
forged_judges_assertion_results = results["forged_judges"]["assertion_results"]

raw_judges = results["raw_judges"]["judge"]
raw_judges_metrics = results["raw_judges"]["alignment_metrics"]
raw_judges_assertion_results = results["raw_judges"]["assertion_results"]

annotation_examples = results["annotation_examples"]
finalized_task_description = results["finalized_task_description"]


In [8]:
def format_alignment_metrics(metrics):
    print("{:<40} {:<40} {:<10} {:<10}".format("Criterion", "Assertion", "Type", "Alignment"))
    print("-" * 100)
    for criterion, criterion_data in metrics.items():
        print("{:<40} {:<40} {:<10} {:<10.2f}".format(
            criterion[:40],
            "OVERALL",
            "",
            criterion_data['criterion_metrics']['alignment']
        ))
        print("-" * 100)
        for assertion, assertion_data in criterion_data['per_assertion'].items():
            print("{:<40} {:<40} {:<10} {:<10.2f}".format(
                "",
                assertion[:40],
                assertion_data['type'],
                assertion_data['alignment']
            ))
        print("-" * 100)

print("Alignment metrics for forged judges:")
format_alignment_metrics(forged_judges_metrics)

print("\nAlignment metrics for raw judges:")
format_alignment_metrics(raw_judges_metrics)

Alignment metrics for forged judges:
Criterion                                Assertion                                Type       Alignment 
----------------------------------------------------------------------------------------------------
Completeness and Clarity of Transformed  OVERALL                                             0.56      
----------------------------------------------------------------------------------------------------
                                         check_completeness_of_categories_in_tran llm        0.44      
                                         test_completeness_of_transformed_notes   code       0.00      
----------------------------------------------------------------------------------------------------
Adherence to Privacy Guidelines          OVERALL                                             0.20      
----------------------------------------------------------------------------------------------------
                                       

In [9]:
raw_judges

AssertionScorer(name=None, description=None, criterion_assertion_map=CriterionAssertionMap(name=None, description=None, criterion_to_assertions=WeaveDict({'Adherence to Instructions and Formatting Guidelines': [PythonAssertion(test_name='test_bullet_point_formatting', code="def test_bullet_point_formatting(self):\n    output = self.output.get('output', '')\n    bullet_points = output.split('\\u2022')[1:]\n    formatted_correctly = all(point.strip() != '' for point in bullet_points)\n    self.assertTrue(formatted_correctly, 'Output should be formatted as a list of bullet points.')", evaluation_type='python'), PythonAssertion(test_name='test_presence_of_all_required_categories', code="def test_presence_of_all_required_categories(self):\n    required_categories = ['Chief complaint', 'History of present illness', 'Physical examination', 'Symptoms experienced by the patient', 'New medications prescribed or changed', 'Follow-up instructions']\n    missing_categories = [category for category 

In [10]:
all_annotation_examples = convert_datapoint_to_example(finalized_task_description, all_data)

In [11]:
evaluation = weave.Evaluation(
    scorers=[forged_judge],
    dataset=all_annotation_examples,
)


final_judge_results = asyncio.run(evaluation.evaluate(predict_passthrough))

Evaluated 1 of 67 examples

Evaluated 2 of 67 examples

Evaluated 3 of 67 examples

Evaluated 4 of 67 examples

Evaluated 5 of 67 examples

...

Evaluated 64 of 67 examples

Evaluated 65 of 67 examples

Evaluated 66 of 67 examples

Evaluated 67 of 67 examples

Evaluation summary
{'final_judge': None, 'model_latency': {'mean': 0.021035347411881632}}

In [12]:
# weave.publish(final_judge, name="final_judge")